In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import numpy as np

In [2]:
mnist = tf.keras.datasets.mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_train = []
scaled_test = []
for i in range(train_X.shape[0]):
  scaled_train.append(scaler.fit_transform(train_X[i]))
for i in range(test_X.shape[0]):  
  scaled_test.append(scaler.fit_transform(test_X[i]))

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [6]:
X_train = np.array(scaled_train)
X_test = np.array(scaled_test)
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [7]:
model = Sequential()
model.add(LSTM(128, input_shape=X_train.shape[1:], activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

In [8]:
optimiser = tf.keras.optimizers.Adam(lr=0.001, decay=1e-5)
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=optimiser,
             metrics=['accuracy'])

In [9]:
import time
start = time.time()
model.fit(X_train,
          train_y,
          epochs=3,
          validation_data=(X_test, test_y), verbose=1)
end = time.time()
print('Net time using P100 GPU : ', end-start, ' secs.')

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 168s 3ms/step - loss: 0.6366 - acc: 0.7883 - val_loss: 0.1599 - val_acc: 0.9528
Epoch 2/3
60000/60000 [==============================] - 165s 3ms/step - loss: 0.1706 - acc: 0.9537 - val_loss: 0.0828 - val_acc: 0.9751
Epoch 3/3
60000/60000 [==============================] - 162s 3ms/step - loss: 0.1101 - acc: 0.9710 - val_loss: 0.0729 - val_acc: 0.9788
Net time using P100 GPU :  496.8233947753906  secs.
